In [7]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "last" # all | last | last_expr | none 

In [9]:
for name in dir():
    if not name.startswith('_'):
        del globals()[name]

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import re, os
import cv2
from PIL import Image
import random


In [11]:
from skimage import data, color
from skimage.transform import rescale, resize, downscale_local_mean

In [ ]:
faults_url = '/Users/bawfeh78/Downloads/images_IL_XL_labels/faults'
for i, fname in enumerate(os.listdir(faults_url)):
    if fname.endswith('png'):
        print(fname)
        img = plt.imread(os.path.join(faults_url, fname))
        dpi = 10
        left, right = 5*dpi, 2*dpi
        bottom, top = 8*dpi, 2*dpi
        img = img[bottom:-top,left:-right]
        plt.axis('off')
        plt.imshow(img)
        plt.savefig(os.path.join(faults_url+'/jpg', fname[:-4]+'.jpg'), bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
faults_url = '/Users/bawfeh78/Downloads/images_IL_XL_labels/faults/jpg'
for i, fname in enumerate(os.listdir(faults_url)):
    if fname.endswith('jpg'):
        jpg_img = plt.imread(os.path.join(faults_url, fname))
        threshold = np.quantile(jpg_img, q=0.01)
        print('{} - threshold = {:<5.1f}'.format(fname, threshold))
        jpg_img = np.where(jpg_img>threshold, 255, 0)
        plt.axis('off')
        plt.imshow(jpg_img)
        plt.savefig(os.path.join(faults_url, fname), bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
seis_url = '/Users/bawfeh78/Downloads/images_IL_XL_labels/seis/'
for i, fname in enumerate(os.listdir(seis_url)):
    if fname.endswith('png'):
        print(fname)
        img = plt.imread(os.path.join(seis_url, fname))
        dpi = 10
        left, right = 5*dpi, 2*dpi
        bottom, top = 8*dpi, 2*dpi
        img = img[bottom:-top,left:-right]
        plt.axis('off')
        plt.imshow(img)
        plt.savefig(os.path.join(seis_url+'/jpg', fname[:-4]+'.jpg'), bbox_inches='tight', pad_inches=0)
plt.show()

In [ ]:
fname = 'IL3600'
seis_url = '/Users/bawfeh78/Downloads/images_IL_XL_labels/seis/jpg'
faults_url = '/Users/bawfeh78/Downloads/images_IL_XL_labels/faults/jpg'
image1 = os.path.join(seis_url, fname+'.jpg')
image2 = os.path.join(faults_url, fname+'_fault.jpg')

img1 = plt.imread(image1)
img2 = plt.imread(image2)
if (img1.shape == img2.shape):
    fig = plt.figure(figsize=(8,8))
    plt.axis('off')
    plt.imshow(np.where(img2==0, img2, img1))
else:
    print('Cropped images have different size!')
plt.show()

In [ ]:
images_dir = sorted(os.listdir(seis_url))
mask_dir = sorted(os.listdir(faults_url))
random.Random(247).shuffle(images_dir)
random.Random(247).shuffle(mask_dir)
print("number of images = {}\nnumber of labels = {}".format(len(images_dir),len(mask_dir)))

In [ ]:
def train_test_val_split(total_len, ratio = (5,1,4)):
    N = sum(ratio)
    delta = total_len * np.divide(ratio, N)
    right = np.clip(np.cumsum(np.round(delta), dtype=int), 0, total_len-1)
    left = np.append([0], right[:-1]+1)
    return list(zip(left, right))


In [ ]:
images_array, mask_array = [], []
for i, m in zip(images_dir, mask_dir):
    if i[:-4] != m[:-10]:
        print(i, m)
    else:
        ipath = os.path.join(seis_url, i)
        images_array.append(plt.imread(ipath))
        mpath = os.path.join(faults_url, m)
        mask_array.append(plt.imread(mpath))

In [ ]:
save_path = '/Users/bawfeh78/Downloads/images_IL_XL_labels/'
splits = train_test_val_split(len(images_array))
left, right = splits[0]
np.savez_compressed(os.path.join(save_path,'train'), 
                    images=images_array[left:right], 
                    labels=mask_array[left:right])
left, right = splits[1]
np.savez_compressed(os.path.join(save_path,'val'), 
                    images=images_array[left:right], 
                    labels=mask_array[left:right])
left, right = splits[2]
np.savez_compressed(os.path.join(save_path,'test'), 
                    images=images_array[left:right], 
                    labels=mask_array[left:right])